In [1]:
%%writefile dashboard.py
class Dashboard:
    @staticmethod
    def show(students):
        print("=== DASHBOARD QUẢN LÝ SINH VIÊN ===")
        print(f"Tổng số sinh viên: {len(students)}")

        if students:
            avg_gpa = sum(s.gpa for s in students) / len(students)
            print(f"GPA trung bình: {avg_gpa:.2f}")
        else:
            print("GPA trung bình: 0")


Writing dashboard.py


In [2]:
%%writefile student.py
class Student:
    def __init__(self, id, name, gpa):
        self.id = id
        self.name = name
        self.gpa = gpa

    def info(self):
        return f"{self.id} - {self.name} - GPA: {self.gpa}"


Writing student.py


In [3]:
%%writefile student_manager.py
from student import Student

class StudentManager:
    def __init__(self):
        self.students = []

    def add_student(self, student):
        self.students.append(student)

    def show_students(self):
        for s in self.students:
            print(s.info())

    def delete_student(self, id):
        self.students = [s for s in self.students if s.id != id]


Writing student_manager.py


In [4]:
%%writefile file_storage.py
import csv
from student import Student

class FileStorage:
    @staticmethod
    def save(filename, students):
        with open(filename, "w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow(["ID", "Name", "GPA"])
            for s in students:
                writer.writerow([s.id, s.name, s.gpa])

    @staticmethod
    def load(filename):
        students = []
        try:
            with open(filename, "r", encoding="utf-8") as f:
                reader = csv.DictReader(f)
                for row in reader:
                    students.append(
                        Student(row["ID"], row["Name"], float(row["GPA"]))
                    )
        except FileNotFoundError:
            pass
        return students


Writing file_storage.py


In [5]:
%%writefile auth.py
import csv
import os

class Auth:
    FILE = "users.csv"

    @staticmethod
    def register(username, password):
        users = []
        if os.path.exists(Auth.FILE):
            with open(Auth.FILE, "r", encoding="utf-8") as f:
                users = list(csv.DictReader(f))

        for u in users:
            if u["username"] == username:
                return False

        users.append({"username": username, "password": password})

        with open(Auth.FILE, "w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=["username", "password"])
            writer.writeheader()
            writer.writerows(users)

        return True

    @staticmethod
    def login(username, password):
        if not os.path.exists(Auth.FILE):
            return False

        with open(Auth.FILE, "r", encoding="utf-8") as f:
            for u in csv.DictReader(f):
                if u["username"] == username and u["password"] == password:
                    return True
        return False


Writing auth.py


In [7]:
from auth import Auth
from student import Student
from student_manager import StudentManager
from file_storage import FileStorage
from dashboard import Dashboard

print("1. Đăng ký")
print("2. Đăng nhập")
choice = input("Chọn: ")

if choice == "1":
    u = input("Username: ")
    p = input("Password: ")
    print("Đăng ký thành công" if Auth.register(u, p) else "Tài khoản tồn tại")

elif choice == "2":
    u = input("Username: ")
    p = input("Password: ")

    if Auth.login(u, p):
        manager = StudentManager()
        manager.students = FileStorage.load("students.csv")

        manager.add_student(Student("SV01", "An", 3.5))
        manager.add_student(Student("SV02", "Bình", 3.8))

        Dashboard.show(manager.students)
        manager.show_students()

        FileStorage.save("students.csv", manager.students)
    else:
        print("Đăng nhập thất bại")


1. Đăng ký
2. Đăng nhập
Chọn: 2
Username: Nhi
Password: nhi
=== DASHBOARD QUẢN LÝ SINH VIÊN ===
Tổng số sinh viên: 2
GPA trung bình: 3.65
SV01 - An - GPA: 3.5
SV02 - Bình - GPA: 3.8
